In [1]:
import requests

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo'

r = requests.get(url)

In [26]:
data = r.json()
data

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close prices and volume',
  '2. Symbol': 'IBM',
  '3. Last Refreshed': '2025-10-02 19:55:00',
  '4. Interval': '5min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2025-10-02 19:55:00': {'1. open': '286.8300',
   '2. high': '286.9400',
   '3. low': '286.8200',
   '4. close': '286.9400',
   '5. volume': '344'},
  '2025-10-02 19:50:00': {'1. open': '286.8500',
   '2. high': '286.9000',
   '3. low': '286.8200',
   '4. close': '286.8200',
   '5. volume': '310'},
  '2025-10-02 19:45:00': {'1. open': '286.8201',
   '2. high': '286.8201',
   '3. low': '286.8200',
   '4. close': '286.8200',
   '5. volume': '10'},
  '2025-10-02 19:40:00': {'1. open': '286.9000',
   '2. high': '286.9000',
   '3. low': '286.8500',
   '4. close': '286.9000',
   '5. volume': '38'},
  '2025-10-02 19:35:00': {'1. open': '286.8200',
   '2. high': '286.8200',
   '3. low': '286.8200',
   '4. close': '286.8200',

In [27]:
import pandas as pd
import datetime
ticker = data['Meta Data']['2. Symbol']
del data['Meta Data']
data = data['Time Series (5min)'] #needlesly nested


In [28]:
df = pd.DataFrame.from_dict(data).T
df = df.apply(pd.to_numeric).reset_index()
df = df.rename(columns={'1. open' : 'open',
           '2. high' : 'high',
           '3. low' : 'low',
           '4. close' :'close',
           '5. volume' : 'volume',
           'index' : 'datetime'})
df

,datetime,open,high,low,close,volume
0,2025-10-02 19:55:00,286.8300,286.9400,286.820,286.940,344
1,2025-10-02 19:50:00,286.8500,286.9000,286.820,286.820,310
2,2025-10-02 19:45:00,286.8201,286.8201,286.820,286.820,10
3,2025-10-02 19:40:00,286.9000,286.9000,286.850,286.900,38
4,2025-10-02 19:35:00,286.8200,286.8200,286.820,286.820,71
...,...,...,...,...,...,...
95,2025-10-02 11:55:00,284.4350,284.5000,284.090,284.225,22444
96,2025-10-02 11:50:00,284.2750,284.6000,284.275,284.380,15992
97,2025-10-02 11:45:00,284.2500,284.3500,284.100,284.275,18083
98,2025-10-02 11:40:00,284.7900,284.9000,284.250,284.320,26827


In [ ]:
def datetime2List(date : datetime.date):
    """
    Turns a datetime object into a list
    """
    out = []
    out.append(date.year)
    out.append()
df[['year', 'month', 'day', 'hour', 'minute', 'second']] = datetime.fromisoformat(df['datetime'])